In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
num_users = 1000
num_days = 14
visit_rate = 0.5
treat_rate = 0.05
geometric_rate = 0.1
p_a = 0.03
p_b = 0.07

In [23]:
is_treat = np.random.choice([0, 1], size = 1000, p = [1 - treat_rate, treat_rate])

In [16]:
assign = pd.Series(users == True)

In [17]:
assign

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [29]:
visits = np.random.choice([0, 1], size = (num_users, num_days), p = [1 - visit_rate, visit_rate])

In [30]:
views = np.zeros([num_users, num_days])

In [34]:
for i in range(num_users):
    for j in range(num_days):
        views[i, j] = np.random.geometric(geometric_rate)

In [35]:
clicks = np.zeros([num_users, num_days])

In [40]:
for i in range(num_users):
    for j in range(num_days):
        if visits[i, j] == 1:
            if is_treat[i] == 1:
                clicks[i, j] = np.random.binomial(n = views[i, j], p = p_b)
            else:
                clicks[i, j] = np.random.binomial(n = views[i, j], p = p_a)
                

In [41]:
# wrangling data

In [43]:
import json

In [44]:
with open('singapore_twitter_copy.json', 'r') as f:
    dat = json.load(f)

In [65]:
#dat

In [48]:
type(dat)

list

In [49]:
len(dat)

13740

In [50]:
dat[0]

{'referenced_tweets': [{'type': 'replied_to', 'id': '1491816184240480284'}],
 'id': '1491816347117916162',
 'in_reply_to_user_id': '204970988',
 'reply_settings': 'everyone',
 'entities': {'mentions': [{'start': 0,
    'end': 12,
    'username': 'QuantoQuant',
    'id': '4332053537'}],
  'annotations': [{'start': 241,
    'end': 249,
    'probability': 0.8172,
    'type': 'Place',
    'normalized_text': 'singapore'}]},
 'text': "@QuantoQuant I don't think we should have death penalty for dealers, but at the very least I think they should be deported if they are here illegally. Alternatively, I think they should go to prison for involuntary manslaughter. \n\nI believe singapore has only had a few dozen put to death.",
 'source': 'Twitter Web App',
 'created_at': '2022-02-10T16:48:28.000Z',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 1,
  'like_count': 2,
  'quote_count': 0},
 'author_id': '204970988'}

In [51]:
sample = dat[0]

In [52]:
type(sample)

dict

In [53]:
len(sample)

10

In [54]:
sample.keys()

dict_keys(['referenced_tweets', 'id', 'in_reply_to_user_id', 'reply_settings', 'entities', 'text', 'source', 'created_at', 'public_metrics', 'author_id'])

In [57]:
rows = []
for treet in dat:
    row = {'author_id' : treet['author_id'],
            'created_at' : treet['created_at']}
    rows.append(row)

In [66]:
#rows

In [59]:
df = pd.DataFrame(rows)

In [60]:
df

author_id                created_at
0                204970988  2022-02-10T16:48:28.000Z
1      1459920226385027086  2022-02-10T16:38:45.000Z
2       840441380652556288  2022-02-10T02:44:41.000Z
3      1388529837833392132  2022-02-10T17:14:16.000Z
4      1197328734417588224  2022-02-10T17:14:14.000Z
...                    ...                       ...
13735  1233528932638642176  2022-02-10T02:36:18.000Z
13736  1363596002100449282  2022-02-10T02:36:15.000Z
13737   893718644084977664  2022-02-10T02:36:01.000Z
13738  1329485154591576074  2022-02-10T02:35:52.000Z
13739             25491333  2022-02-10T01:28:26.000Z

[13740 rows x 2 columns]

In [62]:
df_cnt = df.author_id.value_counts()

In [63]:
df_cnt

author_id
881903898243473409     145
2855329598              43
34565395                42
922256694               38
711460194862018560      36
                      ... 
859424754                1
383179790                1
1410769844836855812      1
1081938217597685760      1
67608195                 1
Name: count, Length: 10557, dtype: int64

In [67]:
# 判断是否有重复的作者：
if (df_cnt > 1).any():
    print('there are repeated authors')
else:
    print('no repeated authors')

there are repeated authors


In [64]:
# yes have repeated authors

In [81]:
from datetime import datetime

In [82]:
date_str = df.created_at[1]

In [83]:
date_str

'2022-02-10T16:38:45.000Z'

In [88]:
date_format = '%Y-%m-%dT%H:%M:%S.000Z'

In [89]:
time1 = datetime.strptime(df.created_at.max(), date_format)

In [90]:
time1

datetime.datetime(2022, 2, 10, 17, 14, 16)

In [91]:
time2 = datetime.strptime(df.created_at.min(), date_format)

In [92]:
time2

datetime.datetime(2022, 2, 10, 1, 28, 26)

In [93]:
#代表最早和最晚的两条twitter之间相隔了多少秒？
time1 - time2

datetime.timedelta(seconds=56750)

In [94]:
# Please wrangle the data so we can look at the number of tweets/retweets for each entity over the different hours in the day.

In [95]:
# get the entities

In [96]:
keys_set = set()

In [100]:
for tweet in dat:
    if 'entities' not in tweet:
        continue
    keys = tweet['entities'].keys()
    keys_str = {str(k) for k in keys}
    keys_set.update(keys_str)

In [101]:
keys_set

{'annotations', 'cashtags', 'hashtags', 'mentions', 'urls'}

In [164]:
entity_types = {'urls': 'url',  'annotations': 'normalized_text', 'hashtags': 'tag', 'chashtag': 'tag', 'mentions': 'id'}

In [146]:
tweet = dat[3]

In [147]:
tweet['entities'].keys()

dict_keys(['urls', 'hashtags', 'mentions'])

In [148]:
tweet['entities']

{'urls': [{'start': 101,
   'end': 124,
   'url': 'https://t.co/XvCwymEI4M',
   'expanded_url': 'http://music.apple.com/fr/album/wannabe/1501233592?i=1501233595&l=en',
   'display_url': 'music.apple.com/fr/album/wanna…'}],
 'hashtags': [{'start': 76, 'end': 81, 'tag': 'ITZY'},
  {'start': 82, 'end': 85, 'tag': '있지'}],
 'mentions': [{'start': 3,
   'end': 17,
   'username': 'chartdataITZY',
   'id': '1424402800054841351'},
  {'start': 86,
   'end': 99,
   'username': 'ITZYofficial',
   'id': '1062578718214770688'}]}

In [149]:
tweet['entities']['hashtags']

[{'start': 76, 'end': 81, 'tag': 'ITZY'},
 {'start': 82, 'end': 85, 'tag': '있지'}]

In [156]:
tweet['entities']['hashtags'][0]['tag']

'ITZY'

In [157]:
[entity_types['hashtags']]

['tag']

In [158]:
k

[{'start': 21, 'end': 31, 'tag': 'TheBatman'}]

In [168]:
bag = []
for tweet in dat:
    tweet_bag = {}
    if 'entities' not in tweet:
        continue
    for entity_type in entity_types:
        if entity_type not in tweet['entities']:
            continue
        list_of_entities_one_type = tweet['entities'][entity_type]
        for entity in list_of_entities_one_type:
            tweet_bag.update({entity[entity_types[entity_type]] : 1})
    bag.append(tweet_bag)

In [187]:
#bag

In [161]:
dff = pd.DataFrame(bag)

In [162]:
dff

singapore  4332053537  Singapore  1382974668  Maldives  796153682  \
0            1.0         1.0        NaN         NaN       NaN        NaN   
1            NaN         NaN        1.0         1.0       NaN        NaN   
2            NaN         NaN        1.0         NaN       1.0        1.0   
3            NaN         NaN        NaN         NaN       NaN        NaN   
4            NaN         NaN        NaN         NaN       NaN        NaN   
...          ...         ...        ...         ...       ...        ...   
13545        NaN         NaN        NaN         NaN       NaN        NaN   
13546        NaN         NaN        NaN         NaN       NaN        NaN   
13547        NaN         NaN        1.0         NaN       NaN        NaN   
13548        NaN         NaN        1.0         NaN       NaN        NaN   
13549        NaN         NaN        NaN         NaN       NaN        NaN   

       https://t.co/XvCwymEI4M  ITZY   있지  1424402800054841351  ...  \
0                          NaN   NaN  NaN                  NaN  ...   
1                          NaN   NaN  NaN                  NaN  ...   
2                          NaN   NaN  NaN                  NaN  ...   
3                          1.0   1.0  1.0                  1.0  ...   
4                          NaN   NaN  NaN                  NaN  ...   
...                        ...   ...  ...                  ...  ...   
13545                      NaN   NaN  NaN                  NaN  ...   
13546                      NaN   NaN  NaN                  NaN  ...   
13547                      NaN   NaN  NaN                  NaN  ...   
13548                      NaN   NaN  NaN                  NaN  ...   
13549                      NaN   NaN  NaN                  NaN  ...   

       1066368503811207169  https://t.co/uPjn42idzF  https://t.co/NbSGPmN7Pm  \
0                      NaN                      NaN                      NaN   
1                      NaN                      NaN                      NaN   
2                      NaN                      NaN                      NaN   
3                      NaN                      NaN                      NaN   
4                      NaN                      NaN                      NaN   
...                    ...                      ...                      ...   
13545                  NaN                      NaN                      NaN   
13546                  NaN                      NaN                      NaN   
13547                  NaN                      NaN                      NaN   
13548                  NaN                      NaN                      NaN   
13549                  NaN                      NaN                      NaN   

       https://t.co/9pjGVzE2Q0  MSSTOPTRENDING  1257000250809581568  \
0                          NaN             NaN                  NaN   
1                          NaN             NaN                  NaN   
2                          NaN             NaN                  NaN   
3                          NaN             NaN                  NaN   
4                          NaN             NaN                  NaN   
...                        ...             ...                  ...   
13545                      NaN             NaN                  NaN   
13546                      NaN             NaN                  NaN   
13547                      NaN             NaN                  NaN   
13548                      NaN             NaN                  NaN   
13549                      NaN             NaN                  NaN   

       https://t.co/QhvOO9BTYw  https://t.co/A1ZcA7zxGt  mr Johnson  \
0                          NaN                      NaN         NaN   
1                          NaN                      NaN         NaN   
2                          NaN                      NaN         NaN   
3                          NaN                      NaN         NaN   
4                          NaN                      NaN         NaN   
.

In [170]:
bag[: 3]

[{'singapore': 1, '4332053537': 1},
 {'Singapore': 1, '1382974668': 1},
 {'Singapore': 1, 'Maldives': 1, '796153682': 1}]

In [180]:
tweet = dat[3]

In [181]:
tweet

{'text': "RT @chartdataITZY: iTunes Top Songs Singapore 🇸🇬 : \n\n#11. 'WANNABE' (NEW) \n\n#ITZY #있지 @ITZYofficial \nhttps://t.co/XvCwymEI4M",
 'id': '1491822838290034693',
 'entities': {'urls': [{'start': 101,
    'end': 124,
    'url': 'https://t.co/XvCwymEI4M',
    'expanded_url': 'http://music.apple.com/fr/album/wannabe/1501233592?i=1501233595&l=en',
    'display_url': 'music.apple.com/fr/album/wanna…'}],
  'hashtags': [{'start': 76, 'end': 81, 'tag': 'ITZY'},
   {'start': 82, 'end': 85, 'tag': '있지'}],
  'mentions': [{'start': 3,
    'end': 17,
    'username': 'chartdataITZY',
    'id': '1424402800054841351'},
   {'start': 86,
    'end': 99,
    'username': 'ITZYofficial',
    'id': '1062578718214770688'}]},
 'created_at': '2022-02-10T17:14:16.000Z',
 'public_metrics': {'retweet_count': 51,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'source': 'Twitter Web App',
 'author_id': '1388529837833392132',
 'reply_settings': 'everyone',
 'referenced_tweets': [{'type': 'retw

In [182]:
type(tweet)

dict

In [184]:
tweet['entities']

{'urls': [{'start': 101,
   'end': 124,
   'url': 'https://t.co/XvCwymEI4M',
   'expanded_url': 'http://music.apple.com/fr/album/wannabe/1501233592?i=1501233595&l=en',
   'display_url': 'music.apple.com/fr/album/wanna…'}],
 'hashtags': [{'start': 76, 'end': 81, 'tag': 'ITZY'},
  {'start': 82, 'end': 85, 'tag': '있지'}],
 'mentions': [{'start': 3,
   'end': 17,
   'username': 'chartdataITZY',
   'id': '1424402800054841351'},
  {'start': 86,
   'end': 99,
   'username': 'ITZYofficial',
   'id': '1062578718214770688'}]}

In [185]:
tweet['referenced_tweets']

[{'type': 'retweeted', 'id': '1491822338874224640'}]

In [188]:
i = 0
for tweet in dat:
    if 'referenced_tweets' not in tweet:
        continue
    if tweet['referenced_tweets'][0]['type'] == 'retweeted':
        i += 1
    

In [189]:
i

8990